# Matrix product operators (MPOs) for representing quantum Hamiltonians

In [1]:
import numpy as np
from scipy import sparse

## Construct MPOs

In [2]:
def construct_ising_hamiltonian_mpo(J, g, L, pbc=False):
    """
    Construct Ising Hamiltonian on a 1D lattice with `L` sites as MPO,
    for interaction parameter `J` and external field parameter `g`.
    """
    # Pauli-X and Z matrices
    X = np.array([[0., 1.], [1.,  0.]])
    Z = np.array([[1., 0.], [0., -1.]])
    I = np.identity(2)
    O = np.zeros((2, 2))
    A = np.array([[I, O, O], [Z, O, O], [-g*X, -J*Z, I]])
    # flip the ordering of the virtual bond dimensions and physical dimensions
    A = np.transpose(A, (2, 3, 0, 1))
    if pbc:
        # periodic boundary conditions:
        # add a direct transition b -> a which applies -J Z at the rightmost lattice site
        AL = np.array([[-g*X, -J*Z, I], [Z, O, O]])
        AR = np.array([[I, -J*Z], [Z, O], [-g*X, O]])
        # flip the ordering of the virtual bond dimensions and physical dimensions
        AL = np.transpose(AL, (2, 3, 0, 1))
        AR = np.transpose(AR, (2, 3, 0, 1))
        return [AL if i == 0 else A if i < L-1 else AR for i in range(L)]
    else:
        return [A[:, :, 2:3, :] if i == 0 else A if i < L-1 else A[:, :, :, 0:1] for i in range(L)]

In [3]:
Alist_ising = construct_ising_hamiltonian_mpo(1.1, 0.7, 5)

In [4]:
# check dimensions (should return True)
Alist_ising[0].shape == (2, 2, 1, 3)

True

In [5]:
# check dimensions (should return True)
Alist_ising[1].shape == (2, 2, 3, 3)

True

In [6]:
# check dimensions (should return True)
Alist_ising[-1].shape == (2, 2, 3, 1)

True

In [7]:
# example
Alist_ising[1]

array([[[[ 1. ,  0. ,  0. ],
         [ 1. ,  0. ,  0. ],
         [-0. , -1.1,  1. ]],

        [[ 0. ,  0. ,  0. ],
         [ 0. ,  0. ,  0. ],
         [-0.7, -0. ,  0. ]]],


       [[[ 0. ,  0. ,  0. ],
         [ 0. ,  0. ,  0. ],
         [-0.7, -0. ,  0. ]],

        [[ 1. ,  0. ,  0. ],
         [-1. ,  0. ,  0. ],
         [-0. ,  1.1,  1. ]]]])

In [8]:
def construct_cluster_hamiltonian_mpo(J, L):
    """
    Construct the cluster state Hamiltonian as MPO.
    """
    # Pauli-X and Z matrices
    X = np.array([[0., 1.], [1.,  0.]])
    Z = np.array([[1., 0.], [0., -1.]])
    I = np.identity(2)
    O = np.zeros((2, 2))
    A = np.array([[I, O, O, O], [Z, O, O, O], [O, X, O, O], [O, O, -J*Z, I]])
    # flip the ordering of the virtual bond dimensions and physical dimensions
    A = np.transpose(A, (2, 3, 0, 1))
    return [A[:, :, 3:4, :] if i == 0 else A if i < L-1 else A[:, :, :, 0:1] for i in range(L)]

In [9]:
Alist_cluster = construct_cluster_hamiltonian_mpo(0.9, 5)

In [10]:
# example: show dimensions
Alist_cluster[0].shape

(2, 2, 1, 4)

In [11]:
# example: show dimensions
Alist_cluster[1].shape

(2, 2, 4, 4)

## Utility functions

In [12]:
def mpo_to_full_tensor(Alist):
    """
    Construct the full tensor corresponding to the MPO tensors `Alist`.

    The i-th MPO tensor Alist[i] is expected to have dimensions (m[i], n[i], D[i], D[i+1]),
    with `m` and `n` the list of logical dimensions and `D` the list of virtual bond dimensions.
    
    The returned tensor has dimensions m[0] x ... x m[L-1] x n[0] x ... x n[L-1]

    Note: Should only be used for debugging and testing.
    """
    # consistency check
    assert Alist[0].ndim == 4
    # use leftmost virtual bond as first dimension
    T = np.transpose(Alist[0], (2, 0, 1, 3))
    # contract virtual bonds
    for i in range(1, len(Alist)):
        T = np.tensordot(T, Alist[i], axes=(-1, 2))
    # contract leftmost and rightmost virtual bond (has no influence if these virtual bond dimensions are 1)
    assert T.shape[0] == T.shape[-1]
    T = np.trace(T, axis1=0, axis2=-1)
    # now T has dimensions m[0] x n[0] x m[1] x n[1] ... m[d-1] x n[d-1];
    # as last step, we group the `m` dimensions together, and likewise the `n` dimensions
    T = np.transpose(T, list(range(0, T.ndim, 2)) + list(range(1, T.ndim, 2)))
    return T

In [13]:
# example
mpo_to_full_tensor([np.random.randn(3, 4, 1, 5), np.random.randn(7, 2, 5, 3), np.random.randn(6, 5, 3, 1)]).shape

(3, 7, 6, 4, 2, 5)

## Construct quantum Hamiltonian as sparse matrix (as reference)

### Transverse-field Ising Hamiltonian

In [14]:
def adjacency_1D_lattice(L, pbc=True):
    """
    Construct the adjacency matrix for a 1D lattice with `L` sites.
    The optional parameter `pbc` specifies whether periodic boundary conditions
    should be used.
    """
    assert L > 1
    # special case
    if L == 2:
        return np.array([[0, 1], [1, 0]])
    if pbc:
        # periodic boundary conditions
        return np.roll(np.identity(L, dtype=int), -1, axis=0) + np.roll(np.identity(L, dtype=int), 1, axis=0)
    else:
        # open boundary conditions
        return np.diag(np.ones(L - 1, dtype=int), k=-1) + np.diag(np.ones(L - 1, dtype=int), k=1)

In [15]:
# should be symmetric
np.linalg.norm(adjacency_1D_lattice(7) - adjacency_1D_lattice(7).T)

0.0

In [16]:
# each site should have 2 neighbors (for periodic boundary conditions)
np.sum(adjacency_1D_lattice(7), axis=0)

array([2, 2, 2, 2, 2, 2, 2])

In [17]:
# example
adjacency_1D_lattice(5, pbc=False)

array([[0, 1, 0, 0, 0],
       [1, 0, 1, 0, 0],
       [0, 1, 0, 1, 0],
       [0, 0, 1, 0, 1],
       [0, 0, 0, 1, 0]])

In [18]:
# Note: this is a solution of Exercise 9.2 (b)
def construct_ising_hamiltonian_sparse(J, g, adj):
    """
    Construct Ising Hamiltonian as sparse matrix,
    for interaction parameter `J` and external field parameter `g`.
    `adj` is the adjacency matrix of the underlying lattice.
    """
    # Pauli-X and Z matrices
    X = sparse.csr_matrix([[0., 1.], [1.,  0.]])
    Z = sparse.csr_matrix([[1., 0.], [0., -1.]])
    # overall number of lattice sites
    L = adj.shape[0]
    H = sparse.csr_matrix((2**L, 2**L), dtype=float)
    for j in range(L):
        for k in range(j+1, L):
            if adj[j, k] > 0:
                H -= J * sparse.kron(sparse.eye(2**j),
                         sparse.kron(Z,
                         sparse.kron(sparse.eye(2**(k-j-1)),
                         sparse.kron(Z,
                                     sparse.eye(2**(L-k-1))))))
    # external field
    for j in range(L):
        H -= g * sparse.kron(sparse.eye(2**j), sparse.kron(X, sparse.eye(2**(L-j-1))))
    return H

In [19]:
adj = adjacency_1D_lattice(5, pbc=False)
Hising = construct_ising_hamiltonian_sparse(1.1, 0.7, adj)
Hising

<32x32 sparse matrix of type '<class 'numpy.float64'>'
	with 180 stored elements in Compressed Sparse Row format>

In [20]:
# convert to NumPy array to show entries
Hising.toarray()

array([[-4.4, -0.7, -0.7, ...,  0. ,  0. ,  0. ],
       [-0.7, -2.2,  0. , ...,  0. ,  0. ,  0. ],
       [-0.7,  0. ,  0. , ...,  0. ,  0. ,  0. ],
       ...,
       [ 0. ,  0. ,  0. , ...,  0. ,  0. , -0.7],
       [ 0. ,  0. ,  0. , ...,  0. , -2.2, -0.7],
       [ 0. ,  0. ,  0. , ..., -0.7, -0.7, -4.4]])

In [21]:
# compare (difference should be zero)
np.linalg.norm(Hising.toarray() - np.reshape(mpo_to_full_tensor(Alist_ising), (32, 32)))

0.0

In [22]:
# periodic boundary conditions
adj = adjacency_1D_lattice(5, pbc=True)
Hising_per = construct_ising_hamiltonian_sparse(1.1, 0.7, adj)
Hising_per

<32x32 sparse matrix of type '<class 'numpy.float64'>'
	with 192 stored elements in Compressed Sparse Row format>

In [23]:
# compare (difference should be zero)
np.linalg.norm(Hising_per.toarray() - np.reshape(mpo_to_full_tensor(construct_ising_hamiltonian_mpo(1.1, 0.7, 5, pbc=True)), (32, 32)))

0.0

### Cluster state Hamiltonian

In [24]:
def construct_cluster_hamiltonian_sparse(J, L):
    """
    Construct the cluster state Hamiltonian as sparse matrix
    on a one-dimensional lattice with open boundary conditions.
    """
    # Pauli-X and Z matrices
    X = sparse.csr_matrix([[0., 1.], [1.,  0.]])
    Z = sparse.csr_matrix([[1., 0.], [0., -1.]])
    H = sparse.csr_matrix((2**L, 2**L), dtype=float)
    h = sparse.kron(sparse.kron(Z, X), Z)
    for j in range(L-2):
        H -= sparse.kron(sparse.eye(2**j),
             sparse.kron(h,
                         sparse.eye(2**(L-j-3))))
    return J*H

In [25]:
Hcluster = construct_cluster_hamiltonian_sparse(0.9, 5)

In [26]:
# compare (difference should be zero)
np.linalg.norm(Hcluster.toarray() - np.reshape(mpo_to_full_tensor(Alist_cluster), (32, 32)))

0.0